In [ ]:
import pandas as pd
import requests
import os
import time
from concurrent.futures import ThreadPoolExecutor

# Load the DataFrame from the Excel file
df = pd.read_excel('../data/horn_bill_telemetry.xlsx')

# Convert 'timestamp' column to datetime if not already
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

# Filter the DataFrame for rows where the timestamp is after '2016-03-02 23:00:00'
df = df[df['timestamp'] > pd.Timestamp('2016-03-02 00:00:00')]

print(f"Filtered data to {len(df)} rows with timestamps after 2016-03-02 23:00:00.")


# Output directory
output_dir = './weather_data/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Output file name
output_file = os.path.join(output_dir, 'final_open_meteo_hourly_weather_data.xlsx')

# Open-Meteo API base URL for hourly data
base_url = "https://api.open-meteo.com/v1/forecast"

# Track progress file
progress_file = os.path.join(output_dir, 'progress.txt')

# Function to fetch hourly data from Open-Meteo API with retries
def fetch_open_meteo_hourly_data(lat, lon, start_datetime, end_datetime, retries=3, delay=2):
    params = {
        "latitude": lat,
        "longitude": lon,
        "start": start_datetime,
        "end": end_datetime,
        "hourly": [
            "temperature_2m", "windspeed_10m", "relative_humidity_2m", "precipitation",
            "dewpoint_2m", "cloudcover", "surface_pressure", "shortwave_radiation",
            "sunshine_duration"
        ],
        "timezone": "UTC"
    }

    for attempt in range(retries):
        try:
            response = requests.get(base_url, params=params, timeout=30)  # Timeout set to 30 seconds
            response.raise_for_status()  # Raises an exception for HTTP errors
            if response.status_code == 200:
                data = response.json().get("hourly", {})
                if data:
                    return data
        except (requests.exceptions.RequestException, ConnectionResetError) as e:
            time.sleep(delay)  # Wait before retrying
    return None

# Function to process a single row
def process_row(index, row):
    print(f"Processing row {index + 1}...")  # Add row index logging

    lat = row['location-lat']
    lon = row['location-long']
    timestamp = row['timestamp']
    start_datetime = timestamp.strftime('%Y-%m-%dT%H:%M:%SZ')
    end_datetime = start_datetime  # Same as start for hourly data

    # Fetch hourly weather data
    weather_data = fetch_open_meteo_hourly_data(lat, lon, start_datetime, end_datetime)
    row_results = []
    if weather_data:
        for hour, data in enumerate(weather_data['time']):
            # print(timestamp)
            row_results.append({
                'event-id': row['event-id'],
                'timestamp': timestamp,  # Convert to datetime
                'location-long': lon,
                'location-lat': lat,
                'temperature': weather_data.get('temperature_2m', [None])[hour],
                'windspeed': weather_data.get('windspeed_10m', [None])[hour],
                'humidity': weather_data.get('relative_humidity_2m', [None])[hour],
                'precipitation': weather_data.get('precipitation', [None])[hour],
                'dewpoint': weather_data.get('dewpoint_2m', [None])[hour],
                'cloud_cover': weather_data.get('cloudcover', [None])[hour],
                'pressure': weather_data.get('surface_pressure', [None])[hour],
                'solar_radiation': weather_data.get('shortwave_radiation', [None])[hour],
                'sunshine_duration': weather_data.get('sunshine_duration', [None])[hour]
            })
    return row_results


# Function to save progress in separate files when row limit exceeds Excel capacity
def save_progress(chunk_results, chunk_index):
    # Check if chunk_results exceeds the row limit for Excel
    max_rows_per_file = 1_048_576
    num_parts = len(chunk_results) // max_rows_per_file + (1 if len(chunk_results) % max_rows_per_file > 0 else 0)

    for part in range(num_parts):
        start_row = part * max_rows_per_file
        end_row = min((part + 1) * max_rows_per_file, len(chunk_results))
        part_df = pd.DataFrame(chunk_results[start_row:end_row])

        # Create a new file for each part
        part_filename = os.path.join(output_dir, f'final_open_meteo_hourly_weather_data_part_{part + 1}.xlsx')
        part_df.to_excel(part_filename, index=False)
        print(f"Saved part {part + 1} to {part_filename}")

    # Save the progress (which chunk is done)
    with open(progress_file, 'w') as f:
        f.write(f"Processed up to chunk {chunk_index}.")

# Check for last processed chunk from the progress file
def get_last_processed_chunk():
    if os.path.exists(progress_file):
        with open(progress_file, 'r') as f:
            last_processed_chunk = f.read().strip()
            return int(last_processed_chunk.split()[-1])
    return -1  # If no progress file, start from the first chunk

# Split the DataFrame into chunks of 5000 rows
chunk_size = 5000
num_chunks = len(df) // chunk_size + (1 if len(df) % chunk_size > 0 else 0)

all_results = []

# Get the last processed chunk to resume from
last_chunk = get_last_processed_chunk()

for i in range(last_chunk + 1, num_chunks):
    print(f"Processing chunk {i + 1} of {num_chunks}...")

    chunk_start = i * chunk_size
    chunk_end = min((i + 1) * chunk_size, len(df))
    chunk = df.iloc[chunk_start:chunk_end]

    # Process the chunk using multithreading
    with ThreadPoolExecutor(max_workers=12) as executor:
        chunk_results = executor.map(lambda item: process_row(*item), chunk.iterrows())

    # Flatten the list of lists and extend to the final results
    for result in chunk_results:
        if result:
            all_results.extend(result)

    # Save the progress after each chunk
    save_progress(all_results, i + 1)
    print(f"Chunk {i + 1} processed.")

# Final save after all chunks
final_df = pd.DataFrame(all_results)
final_df.to_excel(output_file, index=False)
print("All data processed and saved.")

In [1]:
import pandas as pd

# Load the hornbill data and weather data into pandas DataFrames
hornbill_data = pd.read_excel("../data/filtered_hornbill.xlsx")
weather_data = pd.read_excel("part_1.xlsx")

# Find the minimum and maximum event-id in the weather data
min_event_id_weather = weather_data['event-id'].min()
max_event_id_weather = weather_data['event-id'].max()

# Filter the hornbill data to only include rows with event-id between min_event_id_weather and max_event_id_weather
filtered_hornbill_data = hornbill_data[
    (hornbill_data['event-id'] >= min_event_id_weather) & 
    (hornbill_data['event-id'] <= max_event_id_weather)
]

# Select relevant columns from hornbill data
hornbill_filtered_columns = filtered_hornbill_data[['event-id', 'individual-taxon-canonical-name','individual-local-identifier', 'hourly_timestamp', 'location-lat', 'location-long']]

# Select relevant columns from weather data
weather_filtered_columns = weather_data[['event-id','temperature','windspeed','humidity','precipitation','dewpoint','cloud_cover','pressure','solar_radiation','sunshine_duration']]

# Merge the two datasets on 'event-id'
filtered_combined_data = pd.merge(hornbill_filtered_columns, weather_filtered_columns, on='event-id', how='inner')

# Save the filtered combined data to a new Excel file
filtered_combined_data.to_excel("filtered_combined_hornbill_weather_data.xlsx", index=False)

# Print a message indicating successful operation
print("Filtered and combined data saved to 'filtered_combined_hornbill_weather_data.xlsx'")


Filtered and combined data saved to 'filtered_combined_hornbill_weather_data.xlsx'


In [2]:
import pandas as pd

# Load data
data = pd.read_excel('filtered_combined_hornbill_weather_data.xlsx')

# Convert timestamp to datetime and extract the hour
data['hour'] = pd.to_datetime(data['hourly_timestamp']).dt.hour

# Function to filter rows based on hour for each event-id
def filter_by_hour(group):
    return group.iloc[group['hour'].iloc[0] - 1:group['hour'].iloc[0]]

# Apply the filtering for each event-id group
filtered_data = data.groupby('event-id', group_keys=False).apply(filter_by_hour)

# Save the filtered data to Excel
filtered_data.to_excel("part2_final.xlsx", index=False)

C:\Users\visha\AppData\Local\Temp\ipykernel_10668\2626055024.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_data = data.groupby('event-id', group_keys=False).apply(filter_by_hour)


In [5]:
import os
import pandas as pd

# Define the folder containing the Excel files
folder_path = "wd_tem"

# Initialize an empty list to store DataFrames
dataframes = []

# Iterate through all files in the folder
for file in os.listdir(folder_path):
    if file.endswith(".xlsx") or file.endswith(".xls"):
        file_path = os.path.join(folder_path, file)
        
        # Read the Excel file into a DataFrame
        df = pd.read_excel(file_path)
        
        # Check if 'event_id' column exists
        if 'event-id' in df.columns:
            dataframes.append(df)
        else:
            print(f"'event_id' column not found in {file}, skipping.")

# Concatenate all DataFrames into a single DataFrame
if dataframes:
    combined_df = pd.concat(dataframes, ignore_index=True)
    print("All sheets combined successfully.")
    
    # Save the combined DataFrame to a new Excel file
    combined_df.to_excel("combined_data.xlsx", index=False)
    print("Combined data saved to 'combined_data.xlsx'.")
else:
    print("No valid sheets found with 'event_id'.")


All sheets combined successfully.
Combined data saved to 'combined_data.xlsx'.


In [6]:
import pandas as pd

# Path to the already combined Excel file
combined_file = "combined_data.xlsx"

# Load the combined data
try:
    combined_df = pd.read_excel(combined_file)
    print(f"Data loaded successfully from '{combined_file}'.")
except FileNotFoundError:
    print(f"File '{combined_file}' not found. Ensure the file exists and try again.")
    exit()

# Remove duplicate rows based on 'event_id', keeping the first occurrence
if 'event-id' in combined_df.columns:
    deduplicated_df = combined_df.drop_duplicates(subset='event-id', keep='first')
    print("Duplicates based on 'event_id' removed.")
    
    # Save the deduplicated DataFrame to a new Excel file
    deduplicated_file = "combined_deduplicated_data.xlsx"
    deduplicated_df.to_excel(deduplicated_file, index=False)
    print(f"Deduplicated data saved to '{deduplicated_file}'.")
else:
    print("'event-id' column not found in the combined data. Please check the file structure.")


Data loaded successfully from 'combined_data.xlsx'.
Duplicates based on 'event_id' removed.
Deduplicated data saved to 'combined_deduplicated_data.xlsx'.
